# IBM Db2 Event Store - Data Analytics using Python API 

IBM Db2 Event Store is a hybrid transactional/analytical processing (HTAP) system that is designed for IoT workloads. It is empowered by the Db2 Common SQL Engine, the most sophisticated SQL-based analytics query engine available. IBM Db2 Event Store can handle complex queries quickly and efficiently. IBM Db2 Event Store also provides rich data science tooling which allow the arriving data to be quickly and eaily analyzed.

> This Demo is created with IBM Db2 Event Store 2.0 Enterprise edition.

This notebook illustrates how the IBM Db2 Event Store can be integrated with multiple popular scientific tools to perform data analytics.

***Pre-Req: Event_Store_Querying_on_Table***

In [ ]:
# import event store's Python client interface libraries
%matplotlib inline  
from eventstore.common import ConfigurationReader
from eventstore.oltp import EventContext
from eventstore.sql import EventSession
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from scipy import stats
import warnings
import datetime

warnings.filterwarnings('ignore')
plt.style.use("fivethirtyeight")

<a id="connect-to-es"></a>
### 1. Set up connection to IBM Db2 Event Store

**In this demo, we assume your IBM Db2 Event Store is installed with Watson Studio Local (WSL).**

You will need to set the Watson Studio Local's `userID` and `password` that will be used to connect to IBM Db2 Event Store instance.

By default, the connection will be estabilished to the IBM Db2 Event Store instance on the current Watson Studio Local cluster.

For more details on setting up IBM Db2 Event Store connection in Jupyter Notebook, please read the official documentation:
https://www.ibm.com/support/knowledgecenter/en/SSGNPV_2.0.0/dsx/jupyter_prereq.html

## Setting the IP address to connect to your IBM Db2 Event Store cluster

For this, you will need to find out the connection string to your IBM Db2 Event Store cluster.

Perform the following steps:

- Replace the IP address in the below program code with the IP address of your local host
- Then execute the program cell below. It will connect to the IBM Db2 Event Store cluster in the provided connection string. 

In [ ]:
# Using the configuration reader API, set up the userID and password that 
# will be used to connect to IBM Db2 Event Store.

ConfigurationReader.setEventUser("admin")
ConfigurationReader.setEventPassword("password")

### 2. Connect to the database
**IBM Event Store 2.0 instance will by default have a database created with name `EVENTDB`, and the default database `EVENTDB` should not be deleted. Each IBM Event Store 2.0 instance only support exact ONE database.**

In [ ]:
dbName = "EVENTDB"

To run Spark SQL queries, you must set up a Db2 Event Store Spark session. SparkSession is the entry point to programming Spark with the Dataset and DataFrame API. The EventSession class extends the optimizer of the SparkSession class.

In [ ]:
sparkSession = SparkSession.builder.appName("EventStore SQL in Python").getOrCreate()
eventSession = EventSession(sparkSession.sparkContext, dbName)

Now you can execute the command to open the database in the event session you created:

In [ ]:
eventSession.open_database()

### 3. Exploring the database by retrieving all tables
The following code section retrieves the names of all tables that exist in the database.

In [ ]:
with EventContext.get_event_context(dbName) as ctx:
   print("Event context successfully retrieved.")

table_names = ctx.get_names_of_tables()
for idx, name in enumerate(table_names):
   print(name)

Now we have the name of the existing table. We then load the corresponding table and get the data frame references to access the table with query. 

In [ ]:
tabName = "IOT_TEMP"

In [ ]:
tab = eventSession.load_event_table(tabName)
print("Table "+tabName+" successfully loaded.")

Let's recall the table schema we previously created.

In [ ]:
try:
    resolved_table_schema = ctx.get_table(tabName)
    print(resolved_table_schema)
except Exception as err:
    print("Table not found")

And in the following cell we create a temp view with that data frame called `readings` hat we will use in the queries below.

### 4. Data Analytics with IBM Db2 Event Store
Data analytics tasks can be performed on table stored in the IBM Db2 Event Store database with various data analytic tools. 

Let's first take a look at the timestamp range of the record

In [ ]:
tab.createOrReplaceTempView("readings")

In [ ]:
query = "SELECT MIN(ts) MIN_TS, MAX(ts) MAX_TS FROM readings"
print("{}\nRunning query in Event Store...".format(query))
df_data = eventSession.sql(query)
df_data.toPandas()

The following cell converts the timestamps in miliseconds to datetime to make it human readable

In [ ]:
MIN_TS=1541019342393
MAX_TS=1541773999825
print("The time range of the dataset is from {} to {}".format(datetime.datetime.fromtimestamp(MIN_TS/1000).strftime('%Y-%m-%d %H:%M:%S'), 
                                                               datetime.datetime.fromtimestamp(MAX_TS/1000).strftime('%Y-%m-%d %H:%M:%S')))

## Sample Problem
Assume we are only interested in the data recorded by the 12th sensor on the 1st device in the time period on the day of 2018-11-01. And we want to investigate the effects of power consumption and ambient power on the temperature recorded by the sensor in this date.


Because the timestamp is recorded in milisec, we need to convert the datetime of interest to a time range in milisec. And then use the range as a filter in the query.

In [ ]:
start_ts = (datetime.datetime(2018,11,1,0,0) - datetime.datetime(1970,1,1)).total_seconds() * 1000
end_ts = (datetime.datetime(2018,11,2,0,0) - datetime.datetime(1970,1,1)).total_seconds() * 1000
print("The time range of datetime 2018-11-01 in milisec is from {:.0f} to {:.0f}".format(start_ts, end_ts))

IBM Db2 Event Store extends the Spark SQL interface, which allows users to apply filters with SQL queries at ease.  

In the following cell, the relevant data are extracted according to the problem scope. Note that because we are specifying a specific device and sensor, this query is fully exploiting the index defined.

In [ ]:
query = "SELECT * FROM readings WHERE deviceID=1 AND sensorID=12 AND ts >1541030400000 AND ts < 1541116800000 ORDER BY ts"
print("{}\nRunning query in Event Store...".format(query))
refined_data = eventSession.sql(query)
refined_data.createOrReplaceTempView("refined_reading")
refined_data.toPandas()

### Basic Statistics 
For numerical data, knowing the descriptive summary statistics can help a lot in understanding the distribution of the data.  

IBM Event Store extends the Spark DataFrame functionality. We can use the `describe` function to retrieve multiple statistics of table stored in IBM Event Store database.

In [ ]:
refined_data.describe().toPandas()

It's worth noticing that some power reading records are negative, which may be caused by sensor error. The records with negative power reading will be dropped.

In [ ]:
query = "SELECT * FROM readings WHERE deviceID=1 AND sensorID=12 AND ts >1541030400000 AND ts < 1541116800000 AND power > 0 ORDER BY ts"
print("{}\nRunning query in Event Store...".format(query))
refined_data = eventSession.sql(query)
refined_data.createOrReplaceTempView("refined_reading")

Total number of records in the refined table view

In [ ]:
query = "SELECT count(*) count FROM refined_reading"
print("{}\nRunning query in Event Store...".format(query))
df_data = eventSession.sql(query)
df_data.toPandas()

### Covariance and correlation
- Covariance is a measure of how two variables change with respect to each other. It can be examined by calling `.stat.cov()` function on the table.

In [ ]:
refined_data.stat.cov("AMBIENT_TEMP","TEMPERATURE")

In [ ]:
refined_data.stat.cov("POWER","TEMPERATURE")

- Correlation is a normalized measure of covariance that is easier to understand, as it provides quantitative measurements of the statistical dependence between two random variables.  It can be examined by calling `.stat.corr()` function on the table.

In [ ]:
refined_data.stat.corr("AMBIENT_TEMP","TEMPERATURE")

In [ ]:
refined_data.stat.corr("POWER","TEMPERATURE")

### Visualization
Visualization of each feature provides insights of the underlying distributions.

- Distribution of Ambient Temperature

In [ ]:
query = "SELECT ambient_temp FROM refined_reading"
print("{}\nRunning query in Event Store...".format(query))
ambient_temp = eventSession.sql(query)
ambient_temp= ambient_temp.toPandas()
ambient_temp.head()

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(16,6))
stats.probplot(ambient_temp.iloc[:,0], plot=plt.subplot(1,3,1))
axs[1].boxplot(ambient_temp.iloc[:,0])
axs[1].set_title("Boxplot on Ambient_temp")
axs[2].hist(ambient_temp.iloc[:,0], bins = 20)
axs[2].set_title("Histogram on Ambient_temp")

- Distribution of Power Consumption

In [ ]:
query = "SELECT power FROM refined_reading"
print("{}\nRunning query in Event Store...".format(query))
power = eventSession.sql(query)
power= power.toPandas()
power.head()

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(16,6))
stats.probplot(power.iloc[:,0], plot=plt.subplot(1,3,1))
axs[1].boxplot(power.iloc[:,0])
axs[1].set_title("Boxplot on Power")
axs[2].hist(power.iloc[:,0], bins = 20)
axs[2].set_title("Histogram on Power")

- Distribution of Sensor Temperature

In [ ]:
query = "SELECT temperature FROM refined_reading"
print("{}\nRunning query in Event Store...".format(query))
temperature = eventSession.sql(query)
temperature= temperature.toPandas()
temperature.head()

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(16,6))
stats.probplot(temperature.iloc[:,0], plot=plt.subplot(1,3,1))
axs[1].boxplot(temperature.iloc[:,0])
axs[1].set_title("Boxplot on Temperature")
axs[2].hist(temperature.iloc[:,0], bins = 20)
axs[2].set_title("Histogram on Temperature")

- Input-variable vs. Target-variable

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(16,6))
axs[0].scatter(power.iloc[:,0], temperature.iloc[:,0])
axs[0].set_xlabel("power in kW")
axs[0].set_ylabel("temperature in celsius")
axs[0].set_title("Power vs. Temperature")
axs[1].scatter(ambient_temp.iloc[:,0], temperature.iloc[:,0])
axs[1].set_xlabel("ambient_temp in celsius")
axs[1].set_ylabel("temperature in celsius")
axs[1].set_title("Ambient_temp  vs. Temperature")

**By observing the plots above, we noticed:**
- The distribution of power consumption, ambient temperature, and sensor temperature each follows an roughly normal distribution.
- The scatter plot shows the sensor temperature has linear relationships with power consumption and ambient temperature.

## Summary
This demo introduced you to how to perform data analytics with rich tooling using IBM Db2 Event Store.

## Next Step
`"Event_Store_ML_Model_Deployment.ipynb"` will show you how to build machine learning model and deploy the model with IBM Db2 Event Store.